# AlgoBulls Python Developer Coding Assignment (Internshala)

### Design a simple Algorithmic Trading Strategy

###### By NANDAM TEJAS

In [ ]:
!pip install pandas
!pip install numpy
!pip install alpha_vantage
!pip install matplotlib
!pip install plotly
!pip install python-dotenv

!pip freeze > requirements.txt

In [ ]:
#import all modules
import os
from datetime import datetime
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries
import plotly.graph_objects as go

from dotenv import load_dotenv
load_dotenv()

API_KEY = os.environ.get("API_KEY")

### To define a class `ScriptData` which fetches US Stocks using `alpha_vantage` package
##### And the class contains:
- `fetch_intraday_data` `(method arguments: script)`: Which fetches intraday data for given scripts.
- `convert_intraday_data` `(method arguments: script)`: Which converts the fetched data to pandas Dataframe  

In [ ]:
# ScriptData
class ScriptData:
    
    def __init__(self): 
        self.api_key = API_KEY
        self.scripts = {}
        
    def __getitem__(self, script):
        return self.scripts.get(script.upper(), None)
    
    def __setitem__(self, script, output):
        self.scripts[script.upper()] = output
        
    def __contains__(self, script):
        return script.upper() in self.scripts
    
    def fetch_intraday_data(self, script):
        script = script.upper()
        
        if script not in self:
            # Create the TimeSeries and get the intraday data in dictionary
            ts = TimeSeries(key=self.api_key)
            data, meta_data = ts.get_intraday(script)

            # Now store the data in scripts
            self.__setitem__(script, data)
    
    def convert_intraday_data(self, script):
        script = script.upper()
        
        # Get the data from the `fetch_intraday_data` method
        data = self.__getitem__(script)
        
        # If the data is not in dataframe, convert it
        if not isinstance(data, pd.DataFrame):
            # Convert data to pandas Dataframe in `df` 
            df = pd.DataFrame.from_dict({i: data[i]
                                        for i in data.keys()
                                        for j in data[i].keys()},
                                       orient="index")
            # reset column names
            df.columns = list(map(
                lambda x: x.split()[-1],
                df.columns
            ))
            df.reset_index(names=['timestamp'], inplace=True)
        
            self.__setitem__(script, df)

In [ ]:
script_data = ScriptData()

In [ ]:
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data("GOOGL")
script_data['GOOGL']

In [ ]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data("AAPL")
script_data['AAPL']

In [ ]:
'GOOGL' in script_data

In [ ]:
'AAPL' in script_data

In [ ]:
'NVDA' in script_data

### To define function called `indicator1` which takes `df` and `timestamp` as inputs and returns another dataframe with two columns
- `timestamp`: Same as ‘timestamp’ column in ‘df’
- `indicator`: Moving Average of the ‘close’ column in ‘df’. The number of
    elements to be taken for a moving average is defined by ‘timeperiod’. For
    example, if ‘timeperiod’ is 5, then each row in this column will be an average
    of total 5 previous values (including current value) of the ‘close’ column.


In [ ]:
def indicator1(df, timestamp=5):
    """
    - `inputs`: `df` and `timestamp` with default value `5`
    - `returns`: Dataframe of two columns
            - `timestamp`: Same as ‘timestamp’ column in ‘df’
            - `indicator`: Moving Average of the ‘close’ column in ‘df’. The number of
                elements to be taken for a moving average is defined by ‘timeperiod’. For
                example, if ‘timeperiod’ is 5, then each row in this column will be an average
                of total 5 previous values (including current value) of the ‘close’ column.
    """
    df = df.copy(deep=False)
    # create the new column in the given Dataframe
    for i in range(0, df.shape[0]-timestamp+1):
        df.loc[df.index[i+timestamp-1], 'indicator'] = round(
            ((df['close'][i:i+timestamp].astype(np.float64).sum()) / timestamp), 3)

    #return the dataframe with `timestamp` and `indicator` columns
    return df[['timestamp', 'indicator']]

In [ ]:
indicator1(script_data['GOOGL'], timestamp=5)

In [ ]:
indicator1(script_data['AAPL'], timestamp=5)

### To define class `Strategy` which
- Fetch intraday historical day (‘df’) using ScriptData class.
We’ll refer to the ‘close’ column of ‘df’ as close_data.
- Compute indicator data on ‘close’ of ‘df’ using indicator1 function.
We’ll refer to the ‘indicator’ column of this data as indicator_data.
- Generate a pandas DataFrame called ‘signals’ with 2 columns:
    i. ‘timestamp’: Same as ‘timestamp’ column in ‘df’
    ii. ‘signal’: This column can have the following values:
        1. BUY (When: If indicator_data cuts close_data upwards)
        2. SELL (When: If indicator_data cuts close_data downwards)
        3. NO_SIGNAL (When: If indicator_data and close_data don’t cut each other)

In [ ]:
class Strategy:
    
    def __init__(self, script):
        self.script = script
        self.script_data = None
    
    def get_script_data(self):
        """
        Get the script data from `ScriptData` and store it in self.script_data
        """
        script_data = ScriptData()
        script_data.fetch_intraday_data(self.script)
        script_data.convert_intraday_data(self.script)
        self.script_data = script_data[self.script].copy()
        
    def get_signals(self):
        """
        Method used to generate the pandas Dataframe called `signals` with 2 columns:
            - `timestamp`: Same `timestamp` from `self.script_data`
            - `signal`: This column can have following values:
                1. `BUY`
                2. `SELL`
                3. `NO_SIGNAL`
        And prints `signals` Dataframe with only those rows where the signal is either `BUY` or `SELL` 
        """
        # Close data
        close_data = self.script_data[['timestamp', 'close']]
        
        # Indicator data
        indicator_data = indicator1(self.script_data, timestamp=5)
        
        # Create the new Dataframe of name `signals`
        signals = pd.concat([close_data[['timestamp', 'close']], indicator_data[['indicator']]], axis=1)
        
        # Convert close to float64 type
        signals['close'] = signals['close'].astype(np.float64)
        
        # Create diff column in signals Dataframe
        for i in range(0, signals.shape[0]):
            signals.loc[signals.index[i], 'diff'] = np.round(signals['indicator'][i] - signals['close'][i])
        
        # Create `signal` column in signals Dataframe
        flag = np.NaN
        for i in range(0, signals.shape[0]):
            if flag == signals.iloc[i]['diff']:
                set_flag = "NO_SIG"
            else:
                if signals.iloc[i]['indicator'] > signals.iloc[i]['close']:
                    set_flag = "BUY"
                elif signals.iloc[i]['indicator'] < signals.iloc[i]['close']:
                    set_flag = "SELL"
                else:
                    set_flag = "NO_SIG"
                flag = signals.iloc[i]['diff']
            signals.loc[signals.index[i], 'signal'] = set_flag
        
        # print the `signals` Dataframe with `signal` column with [BUY, SELL]
        return signals[(signals['signal'] == 'BUY') | (signals['signal'] == 'SELL')][['timestamp', 'signal']]
            

In [ ]:
strategy = Strategy("NVDA")

In [ ]:
strategy.get_script_data()

In [ ]:
strategy.get_signals()

### [OPTIONAL] Plot a candlestick chart of `df` and `indicator`


In [ ]:
def show_graph(script):
    df = script_data[script.upper()]
    ind = indicator1(df, timestamp=5)

    fig = go.Figure(data=[go.Candlestick(x=df['timestamp'],
                    open=df['open'], high=df['high'],
                    low=df['low'], close=df['close'], 
                    name="Historical Data"), 

                    go.Scatter(x=ind['timestamp'], y=ind['indicator'],
                    line = dict(color = "gray"), name="SMA")
                ]
            )

    fig.update_layout(xaxis_rangeslider_visible=False)

    fig.show()

In [ ]:
show_graph('GOOGL')

In [ ]:
show_graph("AAPL")

In [ ]:
show_graph("NVDA")